In [82]:
from azureml.core import Workspace, Experiment, Environment

ws = Workspace.from_config()
myenv = Environment.get(workspace=ws, name="AzureML-Minimal")
ws.get_details()
#ws = Workspace.get(name="aml-quickstarts-140195")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-140195
Azure region: southcentralus
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-140195


In [83]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

compute_cluster_name = "opt-ml-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing cluster, use this cluster that was found.")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())



Found existing cluster, use this cluster that was found.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 4, 'targetNodeCount': 4, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 4, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-11T03:01:08.430000+00:00', 'errors': None, 'creationTime': '2021-03-11T01:46:50.558844+00:00', 'modifiedTime': '2021-03-11T01:47:05.965571+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [84]:
compute_targets = ws.compute_targets
for compute_name, compute_type in compute_targets.items():
    print(compute_name, compute_type.type, compute_type.provisioning_state)

computemlproject ComputeInstance Succeeded
cpu-cluster AmlCompute Succeeded
opt-ml-cluster AmlCompute Succeeded


In [85]:
from azureml.widgets import RunDetails
from azureml.core.experiment import Experiment
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core.script_run_config import ScriptRunConfig
import os

experiment = Experiment(ws, 'quick-starts-ws-140195')

#cluster = Workspace.compute_targets['udacity-project']

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C" : choice(0.01, 0.1, 1),
    "--max_iter" : choice(20, 40, 60, 100, 150, 200)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = './training', entry_script='train.py', compute_target=compute_target)

#script_run_config.script_run_config.target = cluster

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling = ps,
                                    primary_metric_name = 'accuracy',
                                    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=4,
                                    policy=policy,
                                    estimator= est)

In [89]:
from azureml.widgets import RunDetails
from azureml.core.experiment import Experiment

# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

#hyperdrive_run.get_status()

hyperdrive_run.wait_for_completion(show_output=True)





_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_533acba2-d7a2-4e0e-8c20-b94967b4f606
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_533acba2-d7a2-4e0e-8c20-b94967b4f606?wsid=/subscriptions/d7f39349-a66b-446e-aba6-0053c2cf1c11/resourcegroups/aml-quickstarts-140195/workspaces/quick-starts-ws-140195

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-03-11T03:08:03.433987][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-03-11T03:08:02.642674][API][INFO]Experiment created<END>\n""<START>[2021-03-11T03:08:03.762635][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-03-11T03:08:04.2703119Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_533acba2-d7a2-4e0e-8c20-b94967b4f606
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_533acba2-d7a2-4e0e-8c20-b94967b4f606?wsid=/subscriptions/d7f39

{'runId': 'HD_533acba2-d7a2-4e0e-8c20-b94967b4f606',
 'target': 'opt-ml-cluster',
 'status': 'Canceled',
 'startTimeUtc': '2021-03-11T03:08:01.985358Z',
 'endTimeUtc': '2021-03-11T03:10:05.000378Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name": 

In [88]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

hyperdrive_best_run = hyperdrive_run.get_best_run_by_primary_metric()

print("best run details :", hyperdrive_best_run.get_details())
print('\n')
print("best run file names :", hyperdrive_best_run.get_file_names())
print('\n')
print("best run metrics :", hyperdrive_best_run.get_metrics())

model = hyperdrive_best_run.register_model(model_name='hyperdrive_log_reg_best_run', 
model_path='outputs/model.joblib', tags={"Method" : "HyperDrive"},
properties={"Accuracy" : hyperdrive_best_run.get_metrics()["Accuracy"]})



AttributeError: 'NoneType' object has no attribute 'get_details'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [ ]:
import pandas as pd
from train import clean_data
from sklearn.model_selection import train_test_split
from azureml.core import Dataset

# Use the clean_data function to clean your data.
x, y = clean_data(ds)


all_data = pd.concat([x,y], axis=1)

train_data, test_data = train_test_split(all_data, test_size=0.2, random_state=49)

train_data.to_csv('training/training_df.csv')

ds1= ws.get_default_datastore()
ds1.upload(src_dir='training', target_path='training_data')

train_ds = Dataset.Tabular.from_delimited_files(path=[(ds1, ('training_data/training_df.csv'))])




In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_ds,
    label_column_name='y',
    n_cross_validations=5,
    compute_target=compute_target,
    enable_early_stopping = True,
    enable_onnx_compatible_models = True)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

remote_run = exp.submit(config=automl_config, show_output=True)



In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

automl_best_run.get_metrics()



In [ ]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])

        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators' : list(e[0] for e in step[1].estimators), 'weights' : step[1].weights})
            print()

            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')

        else:
            pprint(step[1].get_params())
            print()

print_model(automl_fitted_model)


In [ ]:
automl_best_run.get_tags()

In [ ]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

automl_best_run_onnx, automl_fitted_model_onnx = remote_run.get_output(return_onnx_model=True)

OnnxConverter.save_onnx_model(automl_fitted_model_onnx, './outputs/best_automl_model.onnx' )